In [66]:
import control as ct
from utils import Position, get_acceleration
import matplotlib.pyplot as plt
import numpy as np

In [67]:
def my_odometry(action, x0, y0, theta0, v0=0, w0=0, dt=0.033):
    """
    Calculate the odometry from the action and the current state.

    :param action: the action to perform
    :param x0: the initial x position
    :param y0: the initial y position
    :param theta0: the initial orientation
    :param v0: the initial linear speed
    :param w0: the initial angular speed
    :param dt: the time step

    :return: (Position, float, float)
    """
    x_dot_dot, w_dot_dot = get_acceleration(action, u=v0, w=w0)

    v1 = v0 + x_dot_dot[0]*dt
    w1 = w0 + w_dot_dot[0]*dt

    # Runge Kutta
    x1 = x0 + v0*dt*np.cos(theta0 + w0*dt/2)
    y1 = y0 + v0*dt*np.sin(theta0 + w0*dt/2)
    theta1 = theta0 + w0*dt

    return Position(x1, y1, theta1), v1, w1

In [68]:
def odom(t, x, u, params):
    dt = params.get('dt', 0.033)

    v0 = u[0]
    w0 = u[1]
    # wl = u[2]
    # wr = u[3]

    x0 = x[0]
    y0 = x[1]
    theta0 = x[2]

    # x_dot_dot, w_dot_dot = get_acceleration([wl, wr], u=v0, w=w0)

    # v1 = v0 + x_dot_dot[0]*dt
    # w1 = w0 + w_dot_dot[0]*dt

    # Runge Kutta
    x1 = x0 + v0*dt*np.cos(theta0 + w0*dt/2)
    y1 = y0 + v0*dt*np.sin(theta0 + w0*dt/2)
    theta1 = theta0 + w0*dt

    return [x1, y1, theta1]

In [69]:
def my_odom_lin(v, w, x0, y0, t0, dt=0.033, v_eq=0, w_eq=0):
    u = np.array([v, w]).T

    B_00 = dt*np.cos(t0+w_eq*dt/2)
    B_01 = -v_eq*dt*np.sin(t0+w_eq*dt/2)*dt/2
    B_10 = dt*np.sin(t0+w_eq*dt/2)
    B_11 = v_eq*dt*np.cos(t0+w_eq*dt/2)*dt/2
    B_20 = 0
    B_21 = w*dt

    B = np.array([[B_00, B_01], [B_10, B_11], [B_20, B_21]])

    A0 = np.array([x0, y0, -t0]).T
    A1 = -B@np.array([v_eq, w_eq]).T

    A = A0 + A1

    x_k = A+B@u

    return x_k

In [70]:
def auto_odom_full(t, x, u, params):
    # Trick is to +1 delay
    dt = params.get('dt', 0.033)

    wl, wr = u[0], u[1]

    x0, y0, theta0, v0, w0 = x[0], x[1], x[2], x[3], x[4]

    x_dot_dot, w_dot_dot = get_acceleration([wl, wr], u=v0, w=w0)
    v1 = v0 + x_dot_dot[0]*dt
    w1 = w0 + w_dot_dot[0]*dt

    # Runge Kutta
    x1 = x0 + v1*dt*np.cos(theta0 + w1*dt/2)
    y1 = y0 + v1*dt*np.sin(theta0 + w1*dt/2)
    theta1 = theta0 + w1*dt

    return [x1, y1, theta1, v1, w1]

In [71]:
io_odom = ct.NonlinearIOSystem(auto_odom_full, None, inputs=('wl', 'wr'), states=('x', 'y', 'th', 'v', 'w'), outputs=('x', 'y', 'th', 'v', 'w'), name='odom', params={'dt': 0.003})

In [72]:
eqpt = ct.find_eqpt(io_odom, [0]*5, [0]*2)
xeq = eqpt[0]

In [73]:
eqpt

(array([0., 0., 0., 0., 0.]), [0, 0])

In [74]:
lin_odom = ct.linearize(io_odom, xeq, 0)

In [75]:
lin_odom.A

array([[1.      , 0.      , 0.      , 0.002955, 0.      ],
       [0.      , 1.      , 0.      , 0.      , 0.      ],
       [0.      , 0.      , 1.      , 0.      , 0.002964],
       [0.      , 0.      , 0.      , 0.985   , 0.      ],
       [0.      , 0.      , 0.      , 0.      , 0.988   ]])

In [76]:
lin_odom.B

array([[ 1.3500e-05,  1.3500e-05],
       [-9.1125e-16,  9.1125e-16],
       [-1.3500e-04,  1.3500e-04],
       [ 4.5000e-03,  4.5000e-03],
       [-4.5000e-02,  4.5000e-02]])

In [77]:
lin_odom.C

array([[1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.]])

In [78]:
lin_odom.D

array([[0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.]])

In [86]:
def linearized_odom(action, x0, y0, theta0, v0=0, w0=0, dt=0.033, return_result=False):
    io_odom = ct.NonlinearIOSystem(auto_odom_full, None, inputs=('wl', 'wr'), states=('x', 'y', 'th', 'v', 'w'), outputs=('x', 'y', 'th', 'v', 'w'), name='odom', params={'dt': dt})
    eqpt = ct.find_eqpt(io_odom, [x0, y0, theta0, v0, w0], [*action], return_result=return_result)
    print(eqpt)
    xeq = eqpt[0]
    lin_odom = ct.linearize(io_odom, xeq, 0)
    x = lin_odom.A@[[xe] for xe in xeq] + lin_odom.B@[[a] for a in action]
    return x.reshape(-1)

In [87]:
linearized_odom([0, 0], 0, 0, 0, v0=0, w0=0, dt=0.033)

(array([0., 0., 0., 0., 0.]), [0, 0])


array([[0.],
       [0.],
       [0.],
       [0.],
       [0.]])

In [89]:
linearized_odom([0, 0], 0.7019999, 1.34470641, 0.0, v0=0, w0=0, dt=0.033)

(array([0., 0., 0., 0., 0.]), [0, 0])


array([[0.],
       [0.],
       [0.],
       [0.],
       [0.]])

In [90]:
auto_odom_full([0, 0], [0.7019999, 1.34470641, 0.0, 0, 0], [0, 0], {'dt': 0.033})

[0.7019999, 1.34470641, 0.0, 0.0, 0.0]